#Installing the neccessary modules needed for sentiment analysis using Textblob

In [21]:
import praw
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import FreqDist
import re
import spacy


#Initialising your reddit credentials. Prior to this, the user has to open an account with reddit and check setting for the neccessary information such as Client id, Client secret and User agent.

In [32]:
reddit = praw.Reddit(client_id=, 
client_secret= 
user_agent=
)

#Choosing the subbreddit you wish to extract comments from and the amount of comments needed.

In [33]:
subreddit = reddit.subreddit('Nickelodeon')

In [35]:
subreddit = reddit.subreddit('Nickelodeon')
for submission in subreddit.hot(limit = 1000):
    print(submission.title)
    print ('Submission ID =', submission.id, '\n')

Fan of old school Nickelodeon? Join us at /r/RetroNickelodeon!
Submission ID = da6wr8 

Come join us in r/AreYouAfraidOfTheDark this Friday at 6PM EST for an AMA with series creator D.J. MacHale!
Submission ID = l506pe 

Danny Phantom: Who is Klemper?
Submission ID = v1oh30 

Where can I watch these FF shows?
Submission ID = v1ni6v 

nickelodeon connected universe (some of it)
Submission ID = v14app 

Is The Loud House a Canadian Show ? Why are there so many fart and burp jokes ?
Submission ID = v1j3f0 

I need help.
Submission ID = v1ixuy 

Six Fanarts #8-Nickelodeon
Submission ID = v0l8aw 

Tak and the Power of Juju timeline
Submission ID = v0mt2q 

trying to find a specific jimmy neutron comic
Submission ID = v008gz 

fungus among us and bowser's inside story
Submission ID = v03y6k 

#6-Jimmy Neutron (Six Fanarts #8)
Submission ID = uzwwif 

Nick Jr. Versions of Inside-Out Boy Safety Queen and Race Rabbit
Submission ID = uzyf12 

#5-Catdog (Six Fanarts #8)
Submission ID = uz8ogo 

C

#Placing the comments in an arrary 

In [36]:
Comment_dict = {'title':[],\
  'score':[],\
      'id':[],\
          'url':[],\
          'comms_num':[],\
            'created':[],\
                  'body':[],\
                       }

In [37]:
for submission in sub_reddit:
    Comment_dict["title"].append(submission.title)
    Comment_dict["score"].append(submission.score)
    Comment_dict["id"].append(submission.id)
    Comment_dict["url"].append(submission.url)
    Comment_dict["comms_num"].append(submission.num_comments)
    Comment_dict["created"].append(submission.created)
    Comment_dict["body"].append(submission.selftext)

#Using panda datframes to hold the comments in a csv file

In [38]:
CNNData = pd.DataFrame(Comment_dict)



#Cleaning the comments and removing common errors found in each line of comment, and finally exporting the clean comment dataset into a csv sheet

In [39]:
def clean(text):
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

In [40]:
CNNData['title'] = CNNData['title'].apply(clean)
CNNData.head()

,title,score,id,url,comms_num,created,body
0,Anybody remember,356,pa29x1,https://v.redd.it/85qrc6k4n4j71,25,1.629733e+09,
1,I m doing a costume a day for all of October T...,213,j71o8m,https://i.redd.it/cve3kg549rr51.jpg,4,1.602112e+09,
2,Nigel Thornberry s Animal World Wild Thornberr...,206,t3tcez,https://www.reddit.com/gallery/t3tcez,40,1.646091e+09,
3,,174,h9aj2n,https://i.redd.it/mr65q1sgp0551.jpg,15,1.592202e+09,
4,Looks like Nickelodeon is going off the air to...,166,guu8rs,https://i.redd.it/b79f6kr2hd251.jpg,67,1.591049e+09,


In [41]:
CNNData.to_csv('NickelodeonComments.csv', index=False) 

#Dropping the columns not needed for analysis

In [86]:

C4chan_data = C4chan_data.drop('body', axis=1)
C4chan_data.head()

,title
0,Melania Trump gets more than she bargained for
1,A message from the moderators of r chan
2,lol git rekt noob
3,THIS IS LIBRARY
4,ISIS decides to attack Brazil


#Applying Textblob analysis, this scores the subjectivity and Polarity in a text and derives a score from this analysis for each line of comment.

In [87]:
from textblob import TextBlob
def getSubjectivity(C4chan_data):
    return TextBlob(C4chan_data).sentiment.subjectivity
def getPolarity(C4chan_data):
        return TextBlob(C4chan_data).sentiment.polarity


#Scoring mechanism for Textblob

In [88]:
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [93]:
final_data = pd.DataFrame(C4chan_data[['title']])

In [94]:
final_data['Subjectivity'] = final_data['title'].apply(getSubjectivity) 
final_data['Polarity'] = final_data['title'].apply(getPolarity) 
final_data['Analysis'] = final_data['Polarity'].apply(analysis)
final_data.head()

,title,Subjectivity,Polarity,Analysis
0,Melania Trump gets more than she bargained for,0.5,0.5,Positive
1,A message from the moderators of r chan,0.0,0.0,Neutral
2,lol git rekt noob,0.7,0.8,Positive
3,THIS IS LIBRARY,0.0,0.0,Neutral
4,ISIS decides to attack Brazil,0.0,0.0,Neutral


#The total count of both negative, positive and neutral scores in the comments

In [95]:
tb_counts = final_data.Analysis.value_counts()

tb_counts

Neutral     709
Positive    183
Negative    104
Name: Analysis, dtype: int64

In [96]:
final_data.to_csv('Final4chanTxtblob.csv', index=False)

#Applying vadersentiment analyser to the comments extracted above

In [97]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [100]:
def vadersentimentanalysis(title):
    vs = analyzer.polarity_scores(title)
    return vs['compound']
final_data['Vader Sentiment + Hatebase'] = final_data['title'].apply(vadersentimentanalysis)

#Scoring Mechanism for Vader sentiment analyser

In [102]:
def vader_analysis(compound):
    if compound >= 0.5:
        return 'Positive'
    elif compound <= -0.5 :
        return 'Negative'
    else:
        return 'Neutral'
final_data['Vader Analysis'] = final_data['Vader Sentiment'].apply(vader_analysis)
final_data.head()

,title,Subjectivity,Polarity,Analysis,Vader Sentiment,Vader Analysis,Vader Sentiment + Hatebase
0,Melania Trump gets more than she bargained for,0.5,0.5,Positive,0.0000,Neutral,0.0000
1,A message from the moderators of r chan,0.0,0.0,Neutral,0.0000,Neutral,0.0000
2,lol git rekt noob,0.7,0.8,Positive,0.3818,Neutral,0.3818
3,THIS IS LIBRARY,0.0,0.0,Neutral,0.0000,Neutral,0.0000
4,ISIS decides to attack Brazil,0.0,0.0,Neutral,-0.4767,Neutral,-0.4767


In [103]:
vader_counts = final_data['Vader Analysis'].value_counts()
vader_counts

Neutral     863
Negative     80
Positive     53
Name: Vader Analysis, dtype: int64

In [104]:
final_data.to_csv('Final4chanVader.csv', index=False)